# Mise à jour de la BDD de NFT
Source: [binance.com](https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1)

In [1]:
import os
import time
import requests
import numpy as np
from utils import *
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

## 1. Ouverture du marketplace

- Paramétrage du navigateur distant
- Chargement de la page d'accueil
- Activation des cookies 

In [12]:
%%time
start_url = 'https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1'
options = webdriver.ChromeOptions()
options.add_argument('--headless')
browser = webdriver.Remote("http://selenium:4444/wd/hub", options=options)
browser.get(start_url)
cookies = browser.find_element_by_xpath("//button[contains(text(),'Accept')]")
cookies.click()

CPU times: user 7.98 ms, sys: 2.92 ms, total: 10.9 ms
Wall time: 48.1 s


## 2. Défilement des pages sur le marketplace
 - Choix du nombre de nft à récupérer
 - Chargement des pages contenant les nfts à récuperer 

In [ ]:
%%time

MAX_NFT_NB = 16#select_number()
CURRENT_PAGE_NUMBER = int(browser.current_url.split("page=")[-1].split("&")[0])
NFT_NUMBER_PER_PAGE = int(browser.current_url.split("rows=")[-1].split("&")[0])
if MAX_NFT_NB/NFT_NUMBER_PER_PAGE == MAX_NFT_NB//NFT_NUMBER_PER_PAGE:
  scroll_down = np.arange((MAX_NFT_NB//NFT_NUMBER_PER_PAGE)-1)
else:
  scroll_down = np.arange(MAX_NFT_NB//NFT_NUMBER_PER_PAGE)
for i in tqdm(scroll_down, postfix=' Scrolling down on the main page'):
  while CURRENT_PAGE_NUMBER == (browser.current_url.split("page=")[-1]).split("&")[0]:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  CURRENT_PAGE_NUMBER = (browser.current_url.split("page=")[-1]).split("&")[0]

## 3. Récupération des liens des pages détaillées
- Détection des boutons donnant accès aux infos détaillées

In [ ]:
%%time

detailed_pages = []
pbar = tqdm(total = MAX_NFT_NB, postfix=" Searching of NFT detailed pages")
BAR_LEVEL = 0
while len(detailed_pages) < MAX_NFT_NB:
  browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  detailed_pages = browser.find_elements_by_xpath('//button[contains(text(),"BSC")]')
  pbar.update(max(0,len(detailed_pages)-BAR_LEVEL))
  BAR_LEVEL = len(detailed_pages)
pbar.close()

## 4. Parcours des pages détaillés
- Récupération des données utiles sur les nft
- Nettoyage des chaînes de caractères
- Stockage des dictionnaires de données
- Fermeture du navigateur distant

In [ ]:
%%time
success = 0
failure = 0
total = len(detailed_pages)

for page in tqdm(detailed_pages, postfix=" Scraping of detailed NFT pages"):
  page.click()
  browser.switch_to.window(browser.window_handles[-1])
  res= parse_nft(browser)
  if res.status_code == 201:
        success += 1
  else:
        failure += 1
  browser.close()
  browser.switch_to.window(browser.window_handles[-1])

print(f"Success={success}/{total}; Failure={failure}/{total} - {res.status_code}")
print(f"NFT collection shape: {requests.get('http://api:8000/nfts/').text.count('_id')}")
browser.quit()

## 5. Automatisation du scraping de nfts
- Lancement du processus toutes les 5h

In [1]:
#%%writefile nft.py

from utils import *

WAIT = 2*60 # 5h = 18000s
MAX_NFT_NB = 16
NEXT_START = datetime.now()+timedelta(seconds=WAIT)
start_url = 'https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1'
browser= None

while True:
    browser = open_binance(start_url, browser) # ouverture du marketplace
    scroll_down(browser, MAX_NFT_NB) # scroll down sur marketplace
    detailed_pages = find_detailed_nft_pages(browser, MAX_NFT_NB) # recherche de pages détaillées
    parse_detailed_nft_pages(browser, detailed_pages) # scraping des pages détaillées
    NEXT_START = wait_next_scraping(NEXT_START, WAIT) # attente avant prochain scraping 

2021-10-01 17:58:18: New scraping initiated
2021-10-01 17:58:18: Selenium configuration
2021-10-01 17:58:33: Binance cookies activation
2021-10-01 17:58:33: Binance page correctly opened


2021-10-01 17:58:33: Scrolling down on the main page: 0it [00:00, ?it/s]
2021-10-01 17:58:33: Searching of NFT detailed pages: 100%|██████████| 16/16 [00:00<00:00, 69.13it/s]
2021-10-01 17:58:33: Scraping of detailed NFT pages: 100%|██████████| 16/16 [01:20<00:00,  5.03s/it]


2021-10-01 17:59:54: Storage on database (Success=16/16; Failure=0/16)
2021-10-01 17:59:54: Waiting for next scraping (24 seconds)
2021-10-01 18:00:18: New scraping initiated
2021-10-01 18:00:23: Binance page correctly opened


2021-10-01 18:00:23: Scrolling down on the main page: 0it [00:00, ?it/s]
2021-10-01 18:00:23: Searching of NFT detailed pages: 100%|██████████| 16/16 [00:02<00:00,  5.80it/s]
2021-10-01 18:00:26: Scraping of detailed NFT pages: 100%|██████████| 16/16 [01:03<00:00,  3.98s/it]


2021-10-01 18:01:29: Storage on database (Success=16/16; Failure=0/16)
2021-10-01 18:01:29: Waiting for next scraping (49 seconds)
2021-10-01 18:02:18: New scraping initiated
2021-10-01 18:02:20: Binance page correctly opened


2021-10-01 18:02:20: Scrolling down on the main page: 0it [00:00, ?it/s]
2021-10-01 18:02:20: Searching of NFT detailed pages: 100%|██████████| 16/16 [00:01<00:00,  8.54it/s]
2021-10-01 18:02:22: Scraping of detailed NFT pages: 100%|██████████| 16/16 [01:03<00:00,  3.95s/it]


2021-10-01 18:03:25: Storage on database (Success=16/16; Failure=0/16)
2021-10-01 18:03:25: Waiting for next scraping (53 seconds)
2021-10-01 18:04:18: New scraping initiated
2021-10-01 18:04:26: Binance page correctly opened


2021-10-01 18:04:26: Scrolling down on the main page: 0it [00:00, ?it/s]
2021-10-01 18:04:26: Searching of NFT detailed pages: 100%|██████████| 16/16 [00:00<00:00, 77.36it/s]
2021-10-01 18:04:26: Scraping of detailed NFT pages: 100%|██████████| 16/16 [00:54<00:00,  3.42s/it]


2021-10-01 18:05:21: Storage on database (Success=16/16; Failure=0/16)
2021-10-01 18:05:21: Waiting for next scraping (57 seconds)


KeyboardInterrupt: 

## 6. Automatisation du scraping de cryptos
- Lancement du processus toutes les 15 minutes

In [1]:
#%%writefile crypto.py

from utils import *

WAIT = 3*60 # 15 min = 900s
NEXT_START = datetime.now()+timedelta(seconds=WAIT)
start_url = 'https://www.binance.com/fr/markets'
browser = None

while True:
    browser = open_binance(start_url, browser, handle_cookies=False) # ouverture de la page principale des cryptos
    nb_pages = find_crypto_pages(browser) # recherche de pages additionnelles
    parse_crypto_pages(browser, nb_pages) # scraping des pages de cryptos
    NEXT_START = wait_next_scraping(NEXT_START, WAIT) # attente avant prochain scraping

2021-10-01 18:16:46: New scraping initiated
2021-10-01 18:16:46: Selenium configuration
2021-10-01 18:16:51: Binance page correctly opened
2021-10-01 18:16:51: Searching of crypto pages


2021-10-01 18:16:53: Scraping of crypto pages: 100%|██████████| 19/19 [00:12<00:00,  1.58it/s]


2021-10-01 18:17:09: Storage on database (Success=366/366; Failure=0/366)


2021-10-01 18:17:09: Waiting for next scraping: 100%|██████████| 158/158 [02:38<00:00,  1.00s/it]


2021-10-01 18:19:47: New scraping initiated
2021-10-01 18:19:50: Binance page correctly opened
2021-10-01 18:19:50: Searching of crypto pages


2021-10-01 18:19:51: Scraping of crypto pages:  42%|████▏     | 8/19 [00:05<00:07,  1.48it/s]


KeyboardInterrupt: 